In [1]:
import chardet
import pandas as pd

In [3]:
import chardet

with open('Dataset/tws.csv', 'rb') as f:
    result = chardet.detect(f.read())

In [5]:
print(result["encoding"])

Windows-1252


In [7]:
df = pd.read_csv("Dataset/tws.csv",encoding=result["encoding"])

In [9]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#Naming headers
df.columns = ["target","ids","date","flag","user","text"]

In [13]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   ids     1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [21]:
df.shape

(1599999, 6)

In [23]:
df.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [25]:
df.target.unique()

array([0, 4], dtype=int64)

In [27]:
df.sample(10)["target"]

825206     4
198057     0
958217     4
934023     4
142904     0
118434     0
1380747    4
1338208    4
196132     0
168542     0
Name: target, dtype: int64

import nltk
nltk.download("all")

In [29]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [31]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

In [33]:
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [35]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # default to noun

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]
    return " ".join(lemmatized_tokens)

df1 = df.copy()
df1["text"] = df1["text"].apply(preprocess_text)

In [37]:
df.head(3)

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


In [39]:
df1.head(3)

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset ca update facebook texting might cry res...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan dive many time ball manage save rest ...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feel itchy like fire


In [41]:
len(df1)

1599999

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

#Feature Extraction
vectorizer = CountVectorizer(max_features= 5000)
X = vectorizer.fit_transform(df1["text"])
y = df1["target"]

#Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [45]:
from sklearn.linear_model import LogisticRegression

In [47]:
#Train model
def train_model(model_class, **kwargs):
    model = model_class(**kwargs)
    model.fit(X_train, y_train)
    return model

Logisticmodel = train_model(LogisticRegression)

In [49]:
#Test model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
def test_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    return accuracy, conf_matrix, class_report

lr_accuracy, lr_conf_matrix, lr_class_report = test_model(Logisticmodel, X_test, y_test)

print(f"Logistic Regression \nAccuracy: {lr_accuracy} \nConfusion Matrix: {lr_conf_matrix} \nClassification Report: {lr_class_report}")

Logistic Regression 
Accuracy: 0.76809375 
Confusion Matrix: [[117694  41800]
 [ 32410 128096]] 
Classification Report:               precision    recall  f1-score   support

           0       0.78      0.74      0.76    159494
           4       0.75      0.80      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [55]:
def polarity_check(polarity):
    if polarity == 0:
        return "Negative"
    elif polarity == 4:
        return "Positive"
    else:
        return 'Neutral'

number = 2
sample = df1["text"][number]
actual_sample = df["text"][number]
test_feature = vectorizer.transform([sample])
prediction = Logisticmodel.predict(test_feature)
target_polarity = polarity_check(prediction[0])
print(f"Polarity of '{actual_sample}' is {target_polarity}")

Polarity of 'my whole body feels itchy and like its on fire ' is Negative


#### Code ends.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [212]:
knn_model = train_model(KNeighborsClassifier)

In [214]:
knn_accuracy, knn_conf_matrix, knn_class_report = test_model(knn_model, X_test, y_test)

KeyboardInterrupt: 

In [217]:
print(f"KNearest Neighbors \nAccuracy: {knn_accuracy} \nConfusion Matrix: {knn_conf_matrix} \nClassification Report: {knn_class_report}")

NameError: name 'knn_accuracy' is not defined

In [ ]:
decisiontree_model = train_model(DecisionTreeClassifier, criterion= "entropy", max_depth= 100)